In [1]:
import pandas as pd
import numpy as np

In [2]:
col_types = {'Reservation Creation Date (DD-MM-YYYY)':str, }
res_df = pd.read_excel('./RequiredData.xlsx', sheet_name="Reservation List")
pay_df = pd.read_excel('./RequiredData.xlsx', sheet_name='Payment List')
res_df.columns, pay_df.columns

(Index(['Reservation ID', 'Reservation Creation Date (MM-DD-YYYY)',
        'Reservation Check-in Date', 'Reservation Check-out Date ',
        'Reservation Total Price ', '#Rooms', 'Cancellation Date',
        'Type\nallotment (1),  sales allotment (2), direct hotel (3)or direct agent(4)'],
       dtype='object'),
 Index(['Payment Date', 'Payment Amount'], dtype='object'))

In [3]:
res_df.columns = ['rid', 'rcr_date', 'ci_date', 'co_date', 'tot_price', 'num_rooms', 'cancel_date', 'res_type']
pay_df.columns = ['p_date', 'p_amount']

In [4]:
res_df.head(10)

,rid,rcr_date,ci_date,co_date,tot_price,num_rooms,cancel_date,res_type
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1
1,2,2023-01-01,2023-05-04,2023-05-09,5000,5,NaT,1
2,3,2023-01-01,2023-08-09,2023-08-10,200,5,2023-01-03,1
3,4,2023-01-02,2023-09-07,2023-09-16,7200,4,2023-01-03,1
4,5,2023-01-02,2023-01-09,2023-01-12,81400,20,NaT,2
5,6,2023-01-03,2023-04-21,2023-04-27,74243,55,NaT,1


In [5]:
def convert_to_datetime(df, inplace=True):
    if not inplace:
        df = df.copy()
    for i in range(len(df.index)):
        for j in range(len(df.columns)):
            if 'date' in df.columns[j]:
                try:
                    df.iloc[i, j] = pd.to_datetime(df.iloc[i, j])
                except:
                    res_df.iloc[i, j] = pd.to_datetime(df.iloc[i, j])
    if inplace:
        return
    else:
        return df

In [6]:
# def convert_to_datetime(df, inplace=True):
#     if not inplace:
#         df = df.copy()
#     for i in range(len(df.index)):
#         for j in range(len(df.columns)):
#             if 'date' in df.columns[j]:
#                 done = False
#                 k = 0
#                 possible_formats = ['%d/%m/%Y', '%d/%m/%Y HH:MM:SS', '%Y/%d/%m', '%Y/%d/%m HH:MM:SS', '%Y/%m/%d', '%Y/%m/%d HH:MM:SS']
#                 while not done:
#                     try:
#                         print(df.iloc[i,j])
#                         df.iloc[i, j] = pd.to_datetime(df.iloc[i, j], format=possible_formats[k])
#                         print(df.iloc[i,j])
#                         done = True
#                     except:
#                         k += 1
#                         if k == len(possible_formats):
#                             break
#     if inplace:
#         return
#     else:
#         return df

In [7]:
def get_cancel_dates(df):
    cancel_dates_to_values = {}
    for index, row in df.iterrows():
        
        cancel_dt = row['cancel_date']
        if cancel_dt is pd.NaT:
            continue
        if cancel_dt not in cancel_dates_to_values.keys():
            cancel_dates_to_values[cancel_dt] = np.array([0, 0]) # res_total_price and numRooms
        cancel_dates_to_values[cancel_dt] += np.array([row['tot_price'], row['num_rooms']])

    return cancel_dates_to_values

In [8]:
def get_cf_df_from_res(df, res_type='all', date_as_index=False):
    df = df.copy()
    if res_type != 'all':
        df = df[df['res_type']==int(res_type)]
    df = df.groupby('rcr_date').sum(numeric_only=True)
    df = df[['tot_price', 'num_rooms']]
    if not date_as_index:
        df = df.reset_index()
    return df

In [9]:
def get_cf_df_from_pay(df, date_as_index=False):
    df = df.copy()
    df = df.groupby('p_date').sum(numeric_only=True)
    df = df[['p_amount']]
    if not date_as_index:
        df = df.reset_index()
    return df

In [10]:
# Define the function to subtract values based on dictionary's key-value mapping
def subtract_values_from_cf_res(row):
    global cancel_dates_to_values
    cur_date = row['rcr_date']
    subtract_array = cancel_dates_to_values.get(cur_date, np.array([0, 0]))
    row[['tot_price', 'num_rooms']] -= subtract_array
    return row

In [11]:
convert_to_datetime(res_df, inplace=True)

In [12]:
res_df.sort_values('rcr_date', inplace=True)

In [13]:
res_df.head(10)

,rid,rcr_date,ci_date,co_date,tot_price,num_rooms,cancel_date,res_type
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1
1,2,2023-01-01,2023-05-04,2023-05-09,5000,5,NaT,1
2,3,2023-01-01,2023-08-09,2023-08-10,200,5,2023-01-03,1
3,4,2023-01-02,2023-09-07,2023-09-16,7200,4,2023-01-03,1
4,5,2023-01-02,2023-01-09,2023-01-12,81400,20,NaT,2
5,6,2023-01-03,2023-04-21,2023-04-27,74243,55,NaT,1


In [14]:
cf_res_df_before_cancel = get_cf_df_from_res(res_df, res_type='all', date_as_index=False)

In [15]:
cf_res_df_before_cancel

,rcr_date,tot_price,num_rooms
0,2023-01-01,29200,20
1,2023-01-02,88600,24
2,2023-01-03,74243,55


In [16]:
# "values" here refer to an np.array() of two values: reservation total price & #rooms
cancel_dates_to_values = get_cancel_dates(res_df)
cancel_dates_to_values

{Timestamp('2023-01-03 00:00:00'): array([7400,    9])}

In [17]:
# Apply the function to subtract values based on dictionary's key-value mapping
cf_res_df = cf_res_df_before_cancel.apply(subtract_values_from_cf_res, axis=1)
cf_res_df

,rcr_date,tot_price,num_rooms
0,2023-01-01,29200,20
1,2023-01-02,88600,24
2,2023-01-03,66843,46


In [18]:
cf_pay_df = get_cf_df_from_pay(pay_df, date_as_index=False)
cf_pay_df

,p_date,p_amount
0,2023-01-01,12000
1,2023-01-05,200
2,2023-01-14,3000
3,2023-03-15,37225
4,2023-09-09,7200
5,2023-12-25,40500


In [19]:
cf_df = pd.merge(cf_res_df, cf_pay_df, left_on='rcr_date', right_on='p_date', how='outer')

In [21]:
for column in cf_df.columns:
    if 'date' not in column:
        cf_df[column].fillna(0, inplace=True)
# data assumption: p_date values must be a subset of rcr_date values 
# & all tot_price values have to be more than zero  
cf_df['cf_percent'] = cf_df['p_amount'] / cf_df['tot_price'] * 100
cf_df

,rcr_date,tot_price,num_rooms,p_date,p_amount,cf_percent
0,2023-01-01,29200.0,20.0,2023-01-01,12000.0,41.09589
1,2023-01-02,88600.0,24.0,NaT,0.0,0.00000
2,2023-01-03,66843.0,46.0,NaT,0.0,0.00000
3,NaT,0.0,0.0,2023-01-05,200.0,inf
4,NaT,0.0,0.0,2023-01-14,3000.0,inf
5,NaT,0.0,0.0,2023-03-15,37225.0,inf
6,NaT,0.0,0.0,2023-09-09,7200.0,inf
7,NaT,0.0,0.0,2023-12-25,40500.0,inf
